# P22 IDS - Complete End-to-End Workflow

This notebook demonstrates the complete workflow from data to deployment.

## Pipeline Steps:
1. Data preparation and preprocessing
2. Hyperparameter tuning
3. Model training
4. Cross-validation
5. Testing and evaluation
6. Deployment-ready inference

## Setup

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

from orchestrator import ServiceOrchestrator
from services.modelTrainingService import ModelTrainingService

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Setup complete")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Phase 1: Data Preparation

In [ ]:
print("=" * 60)
print("PHASE 1: DATA PREPARATION")
print("=" * 60)

# Initialize orchestrator
orchestrator = ServiceOrchestrator('../config.example.yaml')
orchestrator.initialize()

print("\n✓ System initialized")

In [ ]:
# Create or load training dataset
# Option 1: Create synthetic data
np.random.seed(42)
n_samples, n_features, n_classes = 2000, 41, 5

# Generate dataset
features = np.random.randn(n_samples, n_features)
labels = np.random.randint(0, n_classes, n_samples)

# Split into train/test
split_idx = int(0.8 * n_samples)
train_features, test_features = features[:split_idx], features[split_idx:]
train_labels, test_labels = labels[:split_idx], labels[split_idx:]

print(f"\n✓ Dataset created")
print(f"  Training: {train_features.shape}")
print(f"  Testing: {test_features.shape}")
print(f"  Classes: {n_classes}")
print(f"  Features: {n_features}")

## Phase 2: Hyperparameter Optimization

In [ ]:
print("\n" + "=" * 60)
print("PHASE 2: HYPERPARAMETER OPTIMIZATION")
print("=" * 60)

trainer = ModelTrainingService({
    'numEpochs': 50,
    'batchSize': 32,
    'outputDir': '../outputs/training'
})
trainer.start()

print("\nTuning LSTM hyperparameters...")
lstm_tuning = trainer.tuneHyperparameters(
    features=train_features,
    labels=train_labels,
    modelType='lstm',
    nTrials=25
)

print("\n✓ Hyperparameter tuning complete")
print(f"Best score: {lstm_tuning['bestScore']:.4f}")
print(f"Best parameters:")
for k, v in lstm_tuning['bestParams'].items():
    print(f"  {k}: {v}")

## Phase 3: Cross-Validation

In [ ]:
print("\n" + "=" * 60)
print("PHASE 3: CROSS-VALIDATION")
print("=" * 60)

best_params = lstm_tuning['bestParams']

cv_results = trainer.crossValidate(
    features=train_features,
    labels=train_labels,
    modelType='lstm',
    modelConfig={
        'inputSize': n_features,
        'numClasses': n_classes,
        **{k: v for k, v in best_params.items() if k != 'learningRate'}
    }
)

avg = cv_results['averageMetrics']
print(f"\n✓ Cross-validation complete (5 folds)")
print(f"  Accuracy:  {avg['accuracy']:.4f} ± {avg['std_accuracy']:.4f}")
print(f"  Precision: {avg['precision']:.4f}")
print(f"  Recall:    {avg['recall']:.4f}")
print(f"  F1-Score:  {avg['f1Score']:.4f} ± {avg['std_f1Score']:.4f}")

## Phase 4: Final Model Training

In [ ]:
print("\n" + "=" * 60)
print("PHASE 4: FINAL MODEL TRAINING")
print("=" * 60)

training_results = trainer.trainModel(
    features=train_features,
    labels=train_labels,
    modelType='lstm',
    modelConfig={
        'inputSize': n_features,
        'numClasses': n_classes,
        **{k: v for k, v in best_params.items() if k != 'learningRate'}
    }
)

print(f"\n✓ Training complete")
print(f"  Best val accuracy: {training_results['bestValAccuracy']:.4f}")
print(f"  Model saved: {training_results['checkpointPath']}")

## Phase 5: Testing and Evaluation

In [ ]:
print("\n" + "=" * 60)
print("PHASE 5: TESTING AND EVALUATION")
print("=" * 60)

# Load trained model
from services.lstmModelService import LSTMModelService

lstm_service = LSTMModelService({
    'inputSize': n_features,
    'numClasses': n_classes,
    **{k: v for k, v in best_params.items() if k != 'learningRate'},
    'modelPath': training_results['checkpointPath']
})
lstm_service.start()

# Run predictions on test set
test_results = lstm_service.process({
    'features': test_features,
    'metadata': {'source': 'test_set'}
})

predictions = np.array(test_results['predictions'])
confidences = np.array(test_results['confidences'])

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

test_acc = accuracy_score(test_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(
    test_labels, predictions, average='weighted', zero_division=0
)
conf_matrix = confusion_matrix(test_labels, predictions)

print(f"\n✓ Test evaluation complete")
print(f"  Accuracy:  {test_acc:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"  Avg Confidence: {confidences.mean():.4f}")

lstm_service.stop()

In [ ]:
# Visualize confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(n_classes), yticklabels=range(n_classes))
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('../confusion_matrix.png', dpi=150)
plt.show()

print("✓ Confusion matrix saved")

## Phase 6: Production Deployment

In [ ]:
print("\n" + "=" * 60)
print("PHASE 6: PRODUCTION DEPLOYMENT")
print("=" * 60)

# Save deployment configuration
deployment_config = {
    'model_type': 'lstm',
    'model_path': training_results['checkpointPath'],
    'hyperparameters': best_params,
    'performance': {
        'cv_accuracy': float(avg['accuracy']),
        'cv_std': float(avg['std_accuracy']),
        'test_accuracy': float(test_acc),
        'test_f1': float(f1)
    },
    'dataset_info': {
        'n_features': int(n_features),
        'n_classes': int(n_classes),
        'train_samples': int(len(train_labels)),
        'test_samples': int(len(test_labels))
    },
    'deployment_date': datetime.now().isoformat()
}

# Save configuration
deploy_config_path = '../deployment_config.json'
with open(deploy_config_path, 'w') as f:
    json.dump(deployment_config, f, indent=2)

print(f"\n✓ Deployment config saved: {deploy_config_path}")
print("\nDeployment-ready files:")
print(f"  1. Model: {training_results['checkpointPath']}")
print(f"  2. Config: {deploy_config_path}")
print(f"  3. CV Results: {cv_results['resultsPath']}")
print(f"  4. Tuning Results: {lstm_tuning['resultsPath']}")

## Complete Pipeline Summary

In [ ]:
# Create comprehensive summary
summary_df = pd.DataFrame([
    ['Phase 1', 'Data Preparation', f"{n_samples} samples, {n_features} features"],
    ['Phase 2', 'Hyperparameter Tuning', f"Best score: {lstm_tuning['bestScore']:.4f}"],
    ['Phase 3', 'Cross-Validation', f"Accuracy: {avg['accuracy']:.4f} ± {avg['std_accuracy']:.4f}"],
    ['Phase 4', 'Model Training', f"Val accuracy: {training_results['bestValAccuracy']:.4f}"],
    ['Phase 5', 'Testing', f"Test accuracy: {test_acc:.4f}, F1: {f1:.4f}"],
    ['Phase 6', 'Deployment', 'Configuration saved']
], columns=['Phase', 'Task', 'Result'])

print("\n" + "=" * 80)
print("COMPLETE PIPELINE SUMMARY")
print("=" * 80)
print(summary_df.to_string(index=False))

print("\n" + "=" * 80)
print("KEY METRICS")
print("=" * 80)
print(f"Tuning Best Score:     {lstm_tuning['bestScore']:.4f}")
print(f"CV Accuracy:           {avg['accuracy']:.4f} ± {avg['std_accuracy']:.4f}")
print(f"Training Val Accuracy: {training_results['bestValAccuracy']:.4f}")
print(f"Test Accuracy:         {test_acc:.4f}")
print(f"Test F1-Score:         {f1:.4f}")
print(f"Avg Confidence:        {confidences.mean():.4f}")

In [ ]:
# Visualize all results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Training history
history = training_results['trainingHistory']
epochs = range(1, len(history['train_loss']) + 1)
axes[0, 0].plot(epochs, history['train_acc'], 'b-', label='Train')
axes[0, 0].plot(epochs, history['val_acc'], 'r-', label='Val')
axes[0, 0].set_title('Training Progress')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# CV results
fold_results = cv_results['foldResults']
fold_nums = [r['fold'] for r in fold_results]
fold_accs = [r['accuracy'] for r in fold_results]
axes[0, 1].bar(fold_nums, fold_accs, color='steelblue', edgecolor='black')
axes[0, 1].axhline(avg['accuracy'], color='r', linestyle='--', label='Average')
axes[0, 1].set_title('Cross-Validation Results')
axes[0, 1].set_xlabel('Fold')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# Confidence distribution
axes[1, 0].hist(confidences, bins=20, color='lightcoral', edgecolor='black')
axes[1, 0].axvline(confidences.mean(), color='r', linestyle='--', 
                   label=f'Mean: {confidences.mean():.3f}')
axes[1, 0].set_title('Prediction Confidence')
axes[1, 0].set_xlabel('Confidence')
axes[1, 0].set_ylabel('Count')
axes[1, 0].legend()

# Performance comparison
metrics = ['Tuning', 'CV', 'Training', 'Test']
scores = [
    lstm_tuning['bestScore'],
    avg['accuracy'],
    training_results['bestValAccuracy'],
    test_acc
]
colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']
axes[1, 1].bar(metrics, scores, color=colors, edgecolor='black')
axes[1, 1].set_title('Performance Across Stages')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].set_ylim([min(scores) - 0.05, 1.0])
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../pipeline_summary.png', dpi=150)
plt.show()

print("\n✓ Pipeline summary visualizations saved")

## Cleanup

In [ ]:
# Stop all services
trainer.stop()
orchestrator.shutdown()

print("\n" + "=" * 60)
print("✓ COMPLETE END-TO-END PIPELINE FINISHED")
print("=" * 60)
print(f"\nTotal time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nAll outputs saved to: ../outputs/")
print("Deployment configuration: ../deployment_config.json")

## Summary

### Completed Pipeline:
1. ✅ **Data Preparation** - Created/loaded dataset
2. ✅ **Hyperparameter Tuning** - Found optimal parameters
3. ✅ **Cross-Validation** - Validated robustness
4. ✅ **Model Training** - Trained final model
5. ✅ **Testing** - Evaluated on holdout set
6. ✅ **Deployment** - Saved production-ready artifacts

### Output Files:
- Trained model checkpoint
- Deployment configuration
- Training history
- CV results
- Tuning results
- Visualizations

### Next Steps:
- Deploy model using saved checkpoint
- Monitor performance in production
- Retrain periodically with new data
- Compare with CNN model
- Implement ensemble for best results